In [9]:
from google.cloud import storage
import pandas as pd
import numpy as np
import captum

#### gs://marketplace-2xim6sjc/MedMNIST/120/artifactFiles/chestmnist_64.npz

# Initialize a client
client = storage.Client()

# Define the bucket and blob (file) names
bucket_name = 'marketplace-2xim6sjc'
blob_name = 'MedMNIST/120/artifactFiles/chestmnist_64.npz'

# Get the bucket and blob
bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)

# Download the blob to a local file
local_file_path = 'chestmnist_64_local.npz'
blob.download_to_filename(local_file_path)

# Load the .npz file
data = np.load(local_file_path, allow_pickle=True)

# # Access the data
# for key in data:
#     print(f"{key}: {data[key]}")


In [3]:
image = data["train_images"][0]

In [5]:
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Load a pretrained model
model = models.resnet50(pretrained=True)
model.eval()

# Hook the gradients of the target layer
def hook_function(module, grad_in, grad_out):
    global gradients
    gradients = grad_out[0]

# Register hook to the last convolutional layer
target_layer = model.layer4[2].conv3
target_layer.register_backward_hook(hook_function)

# Preprocess the input image
def preprocess_image(img_arr):
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = Image.fromarray(img_arr).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0)
    return img_tensor

# Generate Grad-CAM
def generate_gradcam(img_tensor, model, layer):
    gradcam = captum.attr.GuidedGradCam(model, layer, device_ids=None)
   

# Visualize the Grad-CAM
def visualize_gradcam(img_path, gradcam):
    img = Image.open(img_path).convert('RGB')
    img = np.array(img.resize((224, 224)))
    heatmap = cv2.applyColorMap(np.uint8(255 * gradcam), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img) / 255
    cam = cam / np.max(cam)
    plt.imshow(cam)
    plt.show()

# Example usage
img_tensor = preprocess_image(image)
class_idx = 243  # Example class index
gradcam = generate_gradcam(img_tensor, class_idx)
visualize_gradcam(img_path, gradcam)

Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)


AttributeError: 'Conv2d' object has no attribute 'output'

Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)